<small><i>This notebook was put together by [Abel Meneses-Abad](http://www.meneses-abad.com) for PyData Vancouver 2018. Source and license info is on [GitHub](https://github.com/sorice/nlp_pydata2018/).</i></small>

# Subproceso de Parser de PDF a TXT#

*Códigos elaborados con la versión 20110515 de PDFMiner en Ubuntu 12.04.*

**Fecha de elaboración inicial**: febrero de 2016
**Última actualización**: agosto de 2017

# Resumen

El presente notebook tiene como objetivo presentar varios ejemplos de código para la 
manipulación de archivos PDF utilizando la biblioteca libre de python **PDFMiner**.
Las versiones de los métodos de programación mostrados corresponden a la tarea de
reconstruir la estructura lógica de un documento PDF *no etiquetado* cuando este tiene
dos o más columnas, como es el caso de los artículos científicos, o archivos pdfs de
la prensa y revistas. Los ejemplos son de los autores *Yusuke Shinyama (creador de
PDFMiner), Denis Papathanasiou (colaboradorde PDFMiner) y Abel Meneses (autor del
notebook e investigador de PLN)*. Al mismo tiempo y en secciones cortas al final de
este material se incluyen otros consejos sobre apps o bibliotecas libres útiles para
los que desean procesar PDFs con distintos objetivos: encriptar, portar de versión,
obtener la información de cabecera, etc.

# Índice
<a id='indice'></a>

[Extracción de Textos desde PDF con PDFMiner](#txtextraction_pdfminer)

   - [Ejemplo Parser con PDFMiner](#ejemplo1) Ejemplo con clases y funciones bien explicadas.
Esta primera solución no ordena los textos correctamente cuando tienen más de una 
columna.
      * [Parse Pages v1](#parse_pages2)
      * [Parse LT Objs 2](#parse_lt_objs2)

   - [Solución 2](#Solucion2) Ordenamiento parcial de las columnas, incorrecto cuando
longitudes en x no son correspondientes.
      * [Parse LT Objs 2](#parse_lt_objs2)
      * [Parse Pages v2](#parse_pages3)
  
   - [Análisis final](#Solucion3) Ordenamiento satisfactorio utilizando x0,y0 de los 
TextBox.
      * [Parse LT Objs 3](#parse_lt_objs3)
      * [Parse Pages 3](#parse_pages4)

[Conclusiones](#conclusiones)

[Playfull Parsing with PDFMiner](#playfull_pdfminer)

[Text Extraction at Any Price](#text_extraction)

[How to make standar my PDF: QPDF](#QPDF)

[Ejercicios](#Ejercicios)

[Referencias](#referencias)

[Índice Alfabético](#indice_alfabetico)

Citas format[[1](#Bird2009)]

<a id='txtextraction_pdfminer'></a>
# Extracción de Textos con PDFMiner

Una tarea frecuente, y útil, es extraer la información textual contenida en un PDF.
Generalmente esto es importante cuando se indexan materiales en un buscador, 
o cuando se desean cortar secciones específicas de una serie de PDFs 
- dígase una sección X de una misma revista -, 
o cuando se desea procesar textos ya archivados como tesis, libros, etc para
reutilizar esta información en tareas de NLP como clasificación de archivos en tópicos,
o estudio de citas, etc.
En todos estos casos la **recuperación confiable** del contenido del documento PDF es
esencial para lograr el objetivo con eficacia.

Es importante saber desde el principio, que aunque el formato PDF se ha establecido
como estandar más utilizado de documentos para portar información de un sistema
operativo a otro, este tiene muchas carencias hasta su versión 1.4, aún hoy muy 
utilizada. La existencia de PDFs sin un etiquetado amplio, carentes de información
adicional para extraer *epígrafes, capítulos, anexos, tablas, bibliografía, etc* da
origen a la elaboración de este material.

<a id='ejemplo1'></a>
</br>
## Ejemplo Parser con PDFMiner

Programando la extracción de textos de un PDF con la biblioteca de python: PDFMiner.

[HTML original en un archivo local](./htmls/2.1/Extracting_Text_and_Images_from_PDF_Files/index.html)
, tomado de [URL en Internet](http://denis.papathanasiou.org/posts/2010.08.04.post.html)

In [1]:
from pdfminer.pdfparser import PDFParser, PDFDocument, PDFNoOutlines
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine, LTFigure, LTImage

In [2]:
def with_pdf (pdf_doc, pdf_pwd, fn, *args):
    """Open the pdf document, and apply the function, returning the results"""
    result = None
    try:
        # open the pdf file
        fp = open(pdf_doc, 'rb')
        # create a parser object associated with the file object
        parser = PDFParser(fp)
        # create a PDFDocument object that stores the document structure
        doc = PDFDocument()
        # connect the parser and document objects
        parser.set_document(doc)
        doc.set_parser(parser)
        # supply the password for initialization
        doc.initialize(pdf_pwd)

        if doc.is_extractable:
            # apply the function and return the result
            result = fn(doc, *args)

        # close the pdf file
        fp.close()
    except IOError:
        # the file doesn't exist or similar problem
        pass
    return result

In [3]:
def _parse_toc (doc):
    """With an open PDFDocument object, get the table of contents (toc) data
    [this is a higher-order function to be passed to with_pdf()]"""
    toc = []
    try:
        outlines = doc.get_outlines()
        for (level,title,dest,a,se) in outlines:
            toc.append( (level, title) )
    except PDFNoOutlines:
        pass
    return toc

In [4]:
def get_toc (pdf_doc, pdf_pwd=''):
    """Return the table of contents (toc), if any, for this pdf file"""
    return with_pdf(pdf_doc, pdf_pwd, _parse_toc)

Breve ejemplo que muestra, como en el fichero *txtfield.pdf* contiene los bookmarks:
*Introduction, Referencing the PDF Reference Manual* and *Text State Operators*.

In [5]:
get_toc('test/2.1/txtfield.pdf')

[(1, u'Introduction'),
 (1, u'Referencing the PDF Reference Manual'),
 (1, u'Text State Operators')]

Sin embargo nuestro objetivo es extraer la información de las páginas. El siguiente
método extrae la **estructura, esquema o árbol** de cada página del PDF cargado.
Similar a la imagen de la Figura 1.

<img src="imgs/2.1/layout.png" alt="*" align="center">
<p><caption align="bottom"> </br><em>Figura 1: Objetos layout y su estructura de árbol.</em></caption> 

<p><strong>Nota:</strong> Imagen original de Yusuke Shinyama en 
<a href="http://www.unixuser.org/~euske/python/pdfminer/programming.html" 
        title="Programming with PDFMiner"> PDFMiner Official Documentation</a>

Puedes consultar el [HTML original en un archivo local](./htmls/2.1/Programming_with_PDFMiner/index.html)
donde aparece la imagen mostrada.

<a id='parse_pages1'></a>
</br>
### Parse Pages v1

Implementemos pues un método para extraer los layouts.

In [6]:
def _parse_pages (doc):
    """With an open PDFDocument object, get the pages and parse each one
    [this is a higher-order function to be passed to with_pdf()]"""
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    for page in doc.get_pages():
        interpreter.process_page(page)
        # receive the LTPage object for this page
        layout = device.get_result()
        # layout is an LTPage object which may contain 
        # child objects like LTTextBox, LTFigure, LTImage, etc.

In [7]:
def get_pages (pdf_doc, pdf_pwd=''):
    """Process each of the pages in this pdf file"""
    with_pdf(pdf_doc, pdf_pwd, _parse_pages)

In [8]:
result = get_pages('test/2.1/paper2column.pdf')

In [9]:
print result

None

Volver al [*Índice*](#indice).

<a id='parse_pages2'></a>
</br>
### Parse Pages v1b

Una versión 2 ampliada del *\_parse\_pages* para extraer también los textos dentro de
los objetos que son TextBox.

In [10]:
def _parse_pages (doc, images_folder):
    """With an open PDFDocument object, get the pages, parse each one,
    and return the entire text
    [this is a higher-order function to be passed to with_pdf()]"""
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    # a list of strings, each representing text collected from each page of the doc
    text_content = []
    for i, page in enumerate(doc.get_pages()):
        interpreter.process_page(page)
        # receive the LTPage object for this page
        layout = device.get_result()
        # layout is an LTPage object which may contain 
        # child objects like LTTextBox, LTFigure, LTImage, etc.
        text_content.append(parse_lt_objs(layout, (i+1), images_folder))

    return text_content

en la línea:

    layout = device.get_result()

se obtienen los objetos o segmentos que
contienen las etiquetas del PDF. El problema puntual de este, como puede comprobarse
parseando cualquier pdf que tengáis a mano, es que dichas etiquetas suelen salir de
forma desordenada de acuerdo a la complejidad del texto. Fundamentalmente este problema
es grave en PDFs de 2, 3 o más columnas, o de formas heterogéneas como es el caso de los
periódicos y revistas llevados a formato impreso.

In [11]:
def get_pages (pdf_doc, pdf_pwd='', images_folder='tmp'):
    """Process each of the pages in this pdf file and print the entire text to stdout"""
    return '\n\n'.join(with_pdf(pdf_doc, pdf_pwd, _parse_pages, *tuple([images_folder])))

La función siguiente resuelve cualquier problema de codificación.
Es importante destacar que cualquier objeto *str* solo permite codificación ASCII.
Intente probar 'cadena'.encode('utf8') y comprenderá el error.

In [12]:
def to_bytestring (s, enc='utf8'):
    """Convert the given unicode string to a bytestring, using the standard encoding,
    unless it's already a bytestring"""
    if s:
        if isinstance(s, str):
            return s
        else:
            return s.encode(enc)

<a id='parse_lt_objs1'></a>
</br>
###Parse LT Objs 1###

Una 1ra versión del *parse\_lt\_objs* para extraer todos los objetos textos, figuras 
e imágenes.

In [13]:
def parse_lt_objs (lt_objs, page_number, images_folder, text=[]):
    """Iterate through the list of LT* objects and capture the text or image data contained in each"""
    text_content = [] 

    for lt_obj in lt_objs:
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            # text
            text_content.append(to_bytestring(lt_obj.get_text()))
        elif isinstance(lt_obj, LTImage):
            # an image, so save it to the designated folder, and note it's place in the text 
            saved_file = save_image(lt_obj, page_number, images_folder)
            if saved_file:
                # use html style <img /> tag to mark the position of the image within the text
                text_content.append('<img src="'+os.path.join(images_folder, saved_file)+'" />')
            else:
                print >> sys.stderr, "Error saving image on page", page_number, lt_obj.__repr__
        elif isinstance(lt_obj, LTFigure):
            # LTFigure objects are containers for other LT* objects, so recurse through the children
            text_content.append(parse_lt_objs(lt_obj.objs, page_number, images_folder, text_content))

    return '\n'.join(text_content)

In [14]:
txt = open('test/2.1/text_content.txt','w')
txt.write(str(get_pages('test/2.1/paper2column.pdf')))
txt.close()

Esta solución tiene un problema como se ha explicado la devolución desordenada
de los objetos por la función _parse_page nos dará un texto que contiene errores
estructurales.

Volver al [*Índice*](#indice).

<a id='Solucion2'></a>
</br>
##Solución 2##
**Implementando un organizador de etiquetas según su posición en x y longitud en x.**

Autor: Denis Papathanasiou

In [15]:
def update_page_text_hash (h, lt_obj, pct=0.2):
    """Use the bbox x0,x1 values within pct% to produce lists of associated 
    text within the hash"""
    
    x0 = lt_obj.bbox[0]
    x1 = lt_obj.bbox[2]
    key_found = False
    for k, v in h.items():
        hash_x0 = k[0]
        if x0 >= (hash_x0 * (1.0-pct)) and (hash_x0 * (1.0+pct)) >= x0:
            hash_x1 = k[1]
            if x1 >= (hash_x1 * (1.0-pct)) and (hash_x1 * (1.0+pct)) >= x1:
                # the text inside this LT* object was positioned at the same
                # width as a prior series of text, so it belongs together
                key_found = True
                v.append(to_bytestring(lt_obj.get_text()))
                h[k] = v
    if not key_found:
        # the text, based on width, is a new series,
        # so it gets its own series (entry in the hash)
        h[(x0,x1)] = [to_bytestring(lt_obj.get_text())]
    return h

<a id='parse_lt_objs2'></a>
</br>
###Parse LT Objs 2###

In [16]:
def parse_lt_objs2 (lt_objs, page_number, images_folder, text=[]):
    """Iterate through the list of LT* objects and capture the text or image data contained in each"""
    text_content = [] 

    page_text = {} # k=(x0, x1) of the bbox, v=list of text strings within that bbox width (physical column)
    for lt_obj in lt_objs:
        #text_content.append(str(lt_obj)+'-')
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            # text, so arrange is logically based on its column width
            page_text = update_page_text_hash(page_text, lt_obj)
        elif isinstance(lt_obj, LTImage):
            # an image, so save it to the designated folder, and note it's place in the text 
            saved_file = save_image(lt_obj, page_number, images_folder)
            if saved_file:
                # use html style <img /> tag to mark the position of the image within the text
                text_content.append('<img src="'+os.path.join(images_folder, saved_file)+'" />')
            else:
                print >> sys.stderr, "error saving image on page", page_number, lt_obj.__repr__
        elif isinstance(lt_obj, LTFigure):
            # LTFigure objects are containers for other LT* objects, so recurse through the children
            text_content.append(parse_lt_objs(lt_obj.objs, page_number, images_folder, text_content))

    for k, v in sorted([(key,value) for (key,value) in page_text.items()]):
        # sort the page_text hash by the keys (x0,x1 values of the bbox),
        # which produces a top-down, left-to-right sequence of related columns
        text_content.append('\n'.join(v))

    return '\n'.join(text_content)

<a id='parse_pages3'></a>
</br>
###Parse Pages v2###

In [17]:
def _parse_pages2 (doc, images_folder):
    """With an open PDFDocument object, get the pages, parse each one, and return the entire text
    [this is a higher-order function to be passed to with_pdf()]"""
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    text_content = [] # a list of strings, each representing text collected from each page of the doc
    for i, page in enumerate(doc.get_pages()):
        interpreter.process_page(page)
        # receive the LTPage object for this page
        layout = device.get_result()
        # layout is an LTPage object which may contain child objects like LTTextBox, LTFigure, LTImage, etc.
        text_content.append(parse_lt_objs2(layout, (i+1), images_folder))

    return text_content

def get_pages2 (pdf_doc, pdf_pwd='', images_folder='tmp'):
    """Process each of the pages in this pdf file and print the entire text to stdout"""
    return '\n\n'.join(with_pdf(pdf_doc, pdf_pwd, _parse_pages2, *tuple([images_folder])))

txt = open('test/2.1/text_content2.txt','w')
txt.write(str(get_pages2('test/2.1/paper2column.pdf')))
txt.close()

Como se puede observar esta solución mejora el resultado. Pero particularmente en el 
ejemplo *"test/2.1/paper2column.pdf"*, p.2, la sección *VI.  CIRCUMSTANCES OF PLAGIARISM* 
queda muy mal posicionada debido a una irregularidad en las secciones.

<style>
<!--
P.equacion {font.family: Symbol; text-a1 ig: center)
-->
</style>

<center><strong>Relacionado con la estructura del etiquetado PDF</strong></center></br>
<table border=0 cellspacing=10> 
    <caption align="bottom"> </br><em>Figura 2: Ejemplo visual de las etiquetas 
    contenidas dentro de un pdf.</em>
    </caption> 
<tr align="center">
<th> <img src="imgs/2.1/Real_pdf_LTTextBox_scheme_min.png" alt="*" 
        align="center"> </th>
    <th> </th>
    <td> 
        <p>Sí en <strong>parse_lt_obj</strong> se descomenta la línea: <br><br>

            <center><code>ext_content.append(str(lt_obj)+'-'</code></center></p>
        
        <p>se pueden imprimir los textos con sus respectivos objetos. Ahí notarán que 
            los lt_obj contienen un bbox con la estructura (x0,y0,x1,y1) que definen 
            un box o cuadro de texto. Donde: 
        <p>x0 < x1 (izq-derecha) & y0 < y1 (bottom-top) 
        <p> Generalmente hacia arriba como se muestra en el cuadro A. Y en particular 
            se pueden dar casos de solapamiento como B y C (está compuesto en el orden 
            mostrado solo por los sub-textos C1,C2, y C3). Esta es la razón de la 
            salida con problemas, entre otros. Los cuadros son segmentos reales.</p>
    </td>
</tr>
</table>
    
Volver al [*Índice*](#indice).

<a id='Solucion3'></a>
## Solución 3 Análisis Final
**Implementando un organizador de etiquetas según su posición en x0,y0**

Autor: Abel Meneses

**Breve descripción del problema:** Se trata de organizar automáticamente los textos de 
acuerdo a su posición en x de menor a mayor (o sea de izquierda a derecha), pero 
identificando la existencia de columnas o grupos verticales; y luego organizar cada 
grupo por su **y0** de mayor a menor (o sea de arriba hacia abajo).

In [18]:
def update_page_text_hash (h, lt_obj):
    """Use the bbox x0,y0 values to produce a structured dict without
    any order yet"""
    
    x0 = lt_obj.bbox[0]
    y0 = lt_obj.bbox[1]
    
    k = len(h)
    text = to_bytestring(lt_obj.get_text())
    #I will filter de shortest boxtext like page-number, ISSN, uper-title, etc
    if len(text) > 110:
        h[k] = (x0,y0,text)
    return h

En la función **update\_page\_text\_hash** solo se genera un diccionario con los puntos 
(x0,y0) y como 3er elemento de la tupla el texto obtenido del objeto del árbol del PDF.



Luego son ordenados por las x, y utilizando una táctica de crear grupos con las x que
se diferencien de otras en solo 15° (*ver Figura 3*). Este valor (15°) se obtuvo de forma experimental.
**Importante**: *un objetivo de este algoritmo era no utilizar complejas bibliotecas de
clustering u ordenamiento.* 

%%html
<center><strong>Visión general del ardid matemático empleado.</strong></center></br>
<table border=0 cellspacing=10> 
    <caption align="bottom"> </br><em>Figura 3: Algoritmo de clustering en columnas de 
    los PDF box utilizando los valores de x0,y0.</em>
    </caption> 
<tr align="center">
<th> <img src="imgs/2.1/sameAngle_PDF-box_clustering_min.png" alt="*" 
        align="center"> </th>
    <th> </th>
    <td> 
        <p>En la 3ra versión de <strong>parse_lt_obj</strong> se agrega un segmento que
            utiliza la función <strong>sen(x)</strong>: <br><br>
            
            $$\sin(\alpha)=\frac{(x_{0B}-x_{0A})}{\sqrt{(y_{0B}-y_{0A})^{2}+(x_{0B}-x_{0A})^{2}}}$$
        </p>
        
        <p>Se puede observar que el ángulo  <strong><em>alfa</em></strong>  entre 
            los puntos A y B, sí están 
            en la misma columna, es un ángulo pequeño. Se utilizan los lados del
            triángulo formado por las proyecciones para ilustrar la ecuación del seno.
        <p> El ángulo <strong><em>beta</em></strong> entre los puntos B y D, ubicados 
            en columnas diferentes, es mucho mayor. 
        <p><strong>Nota:</strong> <em>este 
            proceso de cálculo se hace después de ordenar las <strong>x0</strong>, y en el algoritmo se
            comparan cada punto n vs n+1, por lo tanto está garantizado que la 
            situación B vs D ocurre entre puntos que generan un ángulo entre 30° y 90°
            de acuerdo a la relación que hay entre el largo y el ancho de una hoja
            utilizada en los PDFs de textos.</em> </p>
    </td>
</tr>
</table>

Volver al [*Índice*](#indice).

<a id='parse_lt_objs3'></a>
</br>
### Parse LT Objs 3

In [1]:
from math import sqrt

def parse_lt_objs3 (lt_objs, page_number, images_folder, text=[]):
    """Iterate through the list of LT* objects and capture the text or image data contained in each"""
    text_content = [] 

    page_text = {} # k=(x0, x1) of the bbox, v=list of text strings within that bbox width (physical column)
    for lt_obj in lt_objs:
        #text_content.append(str(page_number)+str(lt_obj)+'-')
        if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
            # get x0,y0,text in page_text dictionary
            page_text = update_page_text_hash(page_text, lt_obj)

    pt, k = page_text, []
    
    for i in pt:
        k.append(pt[i])
    #Ordering taking into account x value from min to max.
    k.sort()
    
    ant = (k[0][0],0) #will keep the last x,y checked
    g = 1;group = {}
    
    #Order gropus of x by major y
    for i in range(len(k)):
        #math sen formula of a triangle angle: opose side over left side.
        sen = float(k[i][0]-ant[0])/(sqrt((k[i][1]-ant[1])**2+(k[i][0]-ant[0])**2)+1)
        
        if sen < 0.25: #if angle < 15°, or same column
            group[g],ant = cluster(group,g,i,k)
        
        else: #not in same column
            g += 1 #add new group
            group[g],ant = cluster(group,g,i,k)
      
    #Order every group based on y0 from max to min
    for i in group.keys():
        L = group[i]
        L.sort(reverse=True)
        group[i] = L
        
    for i in group.keys():
        for j in range(len(group[i])):
            text_content.append(''.join(group[i][j][2]))

    return '\n'.join(text_content)

SyntaxError: invalid syntax (<ipython-input-1-4fc52e21dd3b>, line 1)

Método auxiliar par reducir código en la función **parse\_lt\_objs3**.

In [25]:
def cluster(group, g, i, k):
    if g in group.keys():
        temp = group[g]
        temp.append((k[i][1],k[i][0],k[i][2]))
    else: temp = [(k[i][1],k[i][0],k[i][2])]
    return temp, (k[i][0],k[i][1])

Ahora corremos el algoritmo, utilizando funciones actualizadas para este ejemplo del
**\_parse\_pages** y **get\_pages**.

<a id='parse_pages4'></a>
</br>
### Parse Pages v3

In [26]:
def _parse_pages3 (doc, images_folder):
    """With an open PDFDocument object, get the pages, parse each one, and return the entire text
    [this is a higher-order function to be passed to with_pdf()]"""
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    text_content = [] # a list of strings, each representing text collected from each page of the doc
    for i, page in enumerate(doc.get_pages()):
        interpreter.process_page(page)
        # receive the LTPage object for this page
        layout = device.get_result()
        # layout is an LTPage object which may contain child objects like LTTextBox, LTFigure, LTImage, etc.
        text_content.append(parse_lt_objs3(layout, (i+1), images_folder))

    return text_content

def get_pages3 (pdf_doc, pdf_pwd='', images_folder='tmp'):
    """Process each of the pages in this pdf file and print the entire text to stdout"""
    return '\n\n'.join(with_pdf(pdf_doc, pdf_pwd, _parse_pages3, *tuple([images_folder])))

txt = open('test/2.1/text_content3.txt','w')
txt.write(str(get_pages3('test/2.1/paper2column.pdf')))
txt.close()

Volver al [*Índice*](#indice).

<a id='conclusiones'></a>
</br>
# Conclusiones

El resultado es genial tras unas 12.5h de desarrollo y testing. Y tras una semana de
buscar un diseño coherente.
El valor de $sin(\alpha) \leq$ 0.25, ó $\alpha \leq$ 15°, funciona mejor en los 
resultados de ambos documentos. 
Esta es la 3ra versión de este algoritmo que bajó de 97 líneas a 57 y una función 
auxiliar de 6. Total 63 LOC.

Los problemas finales que puede haber se deben a una generación defectuosa del PDF.
Esto se corrige utilizando herramientas que generen PDFs etiquetados en su estandar
1.7, que permite guardar las etiquetas del documento.

Volver al [*Índice*](#indice).

<a id='homework'></a>
</br>
# Homework

Este es el artículo original del creador de PDFMiner Yusuke Shinyama. Sin embargo es
importante saber que los códigos están orientados a la versión actual(2015) de PDFMiner
y que por lo tanto los módulos y métodos de PDFMiner utilizados en el ejemplo de 
análisis son distintos a estos.

[URL en Internet](http://www.unixuser.org/~euske/python/pdfminer/programming.html)
, y [HTML original en un archivo local](./htmls/2.1/Programming_with_PDFMiner/index.html).

<a id='playfull_pdfminer'></a>
</br>
# Playfull Parsing with PDFMiner

Este ejemplo nos permite parsear una tonelada de PDFs y buscar cierta información en él.
En particular el ejemplo está elaborado para extraer la fecha de los informes escrita
dentro de ellos y no en la cabecera. Esto permite identificar en una serie de documentos
cuales tengo y cuales me faltan por su fecha.

[URL en Internet](https://quantcorner.wordpress.com/2014/03/16/parsing-pdf-files-with-python-and-pdfminer/)
, y [HTML original en un archivo local](./htmls/2.1/Parsing_pdf_files_with_Python_and_PDFMiner_-_Quant_Corner/index.html).

Volver al [*Índice*](#indice).

<a id='text_extraction'></a>
</br>
# Text Extraction at Any Price

¿Y si PDFMiner no puede leer el PDF?
Pues bien si esto ocurre puede utilizar la biblioteca python-textract. Esta lib contiene
métodos para leer los PDFs incluso utilizando TSERACT y PDFMiner para cada situación 
adecuadamente. El autor de este notebook les recomienda el material linkeado a 
continuación.

[URL en Internet](http://textract.readthedocs.org/en/latest/python_package.html),
y [HTML original en un archivo local](./htmls/2.1/Python_package_textract_140_documentation/index.html)
    
Volver al [*Índice*](#indice).

<a id='QPDF'></a>
</br>
# How to make standar my PDF: QPDF

Si deseas hacer tu PDF estandar al haber sido generado con cualquier herramienta,
o deseas cambiar su versión, o poner un password, entre otras operaciones te 
recomendamos leer la documentación de QPDF.

[URL en Internet](http://textract.readthedocs.org/en/latest/python_package.html),
y [HTML original en un archivo local](./htmls/2.1/QPDF_Manual/QPDF Manual.html)

Volver al [*Índice*](#indice).

<a id='Ejercicios'></a>
</br>
# Ejercicios

1. Diseñe un parser que extraiga las notas de un pdf generado con el estandar 1.7.
2. Implemente un parser que extraiga las imágenes y las coloque en una carpeta.
3. Escriba un notebook que ejemplifique como insertar diferentes campos útiles en el etiquetado. Ejemplos: pies de páginas, encabezados, números de páginas y notas.
4. Extraiga de una colección de libros los autores de cada uno y el año en que fue escrito.
5. Utilizando la herramienta pdftk extraiga los bookmarks del PDF.
6. Utilizando la herramienta ghostscript extraiga el texto de las páginas e inyecte notas y bookmarks.
7. Basado en los problemas abiertos de Papathaneous:

   a. Implemente un parser que utilice un mecanismo de OCR para hacer el Column Merging, y resolver así problemas existentes en el PDF generados durante su construcción.
   
   b. Cree un script que extraiga las imágenes y las coloque en una carpeta con varios formatos conocidos ppm, gif, jpeg, png.
   
   c. Elabore un script que corrija los problemas en el reconocimiento de la capitalización en los títulos y los epígrafes.
   
   d. Escriba un parser que genere un .xml donde se etiqueten de forma diferenciada el título del documento, autores y secciones del mismo con su fuente.
   
   e. Elabore un script que elimine los números de página sin importar si están al inicio
o al final de la página.
   
   f. Implemente un script que extraiga en una tupla todos los footnotes, su págna correspondiente, y el texto que hace referencia a él dentro del documento.
   
8. Experimente como extraer la información de la cabecera del PDF con la herramienta *popler-utils* y luego con PDFMiner. ¿Cuál es más rápida?

<a id='referencias'></a>
## Referencias

<a id='Bird2009'></a>
[1] *[Bird2009]* Steven Bird, Ewan Klain y Edward Loper,. 
Book **Natural Language Processing with Python**. 2009. 
p. 10 **ISBN**: 978-0-596-51649-9

<a id='Wiki2016'></a>
[2] *[Wiki2016]* Colectivo de autores. 
Enciclopedia Article **Portable Document Format**. 2016. 
[URL en Internet](https://en.wikipedia.org/wiki/Portable_Document_Format#Technical_overview),
[URL local](./htmls/2.1/Portable_Document_Format_Wikipedia_the_free_encyclopedia/index.html)

<a id='Arnold2014'></a>
[3] *[Arnold2014]* Tom Arnold. 
Web Page **Manipulating PDFs with Python**. 2014. 
[URL en Internet](https://www.binpress.com/tutorial/manipulating-pdfs-with-python/167)
[URL local](./htmls/2.1/Manipulating_PDFs_with_Python_Tutorial_Binpress/index.html)

<a id='Lukan2012'></a>
[4] *[Lukan2012]* Dejan Lukan. 
Web Page **PDF File Format: Basic Structure**. 2012.
[URL en Internet](http://resources.infosecinstitute.com/category/exploit-development)
[URL local](./htmls/2.1/PDF_File_Format_Basic_Structure_InfoSec_Resources/index.html)

<a id='alphabetic_index'></a>
## Índice Alfabético

<a id='token'></a>
**Token**: señal, indicio, muestra. Se usa generalmente para referirse a la unidad
más pequeña de procesamiento: palabras, fonemas, n-grams, etc..


Volver al [*Índice*](#indice).